### Imports

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, clear_output
import sys
import time
import os
import time
%matplotlib inline

### Importing training data

In [12]:
dir_path = (os.getcwd() + "\\").replace("\\","/") # If it does not work change it to path where data is stored.
print("Working directory is : ", dir_path)

Working directory is :  /media/abhikcr/New Volume/Study/M.tech 1st Sem/Into to ML/project/Project/Models/Bonsai/


In [17]:
#Loading Pre-processed dataset for Bonsai
dirc = dir_path + '/../../Datasets/iris/'
print(os.listdir(dirc))

['preprocessed iris.zip', 'X_test.npy', 'X_train.npy', 'Y_test.npy', 'Y_train.npy']


In [40]:
Xtrain = np.load(dirc + 'X_train.npy')
Ytrain = np.load(dirc + 'Y_train.npy')
Xtest = np.load(dirc + 'X_test.npy')
Ytest = np.load(dirc + 'Y_test.npy')

In [26]:
from sklearn.preprocessing import LabelEncoder as LE 
from sklearn.preprocessing import OneHotEncoder as OHE

In [27]:
mo1 = LE()
mo2 = OHE()

In [41]:
Ytrain = mo2.fit_transform(mo1.fit_transform((Ytrain)).reshape(-1,1)).todense()

In [42]:
Ytest = mo2.transform(mo1.transform((Ytest)).reshape(-1,1)).todense()

In [43]:
Xtrain.shape, Xtest.shape,Ytrain.shape,Ytest.shape

((112, 4), (38, 4), (112, 3), (38, 3))

In [45]:
# N, dDims = X_train.shape
N, dDims = Xtrain.shape
# nClasses = len(np.unique(Y_train))
nClasses = Ytrain.shape[1]
print('Training Size:',N,',Data Dims:', dDims,',No. Classes:', nClasses)

Training Size: 112 ,Data Dims: 4 ,No. Classes: 3


In [46]:
class Bonsai():
    def __init__(self, nClasses, dDims, pDims, tDepth, sigma, W=None, T=None, V=None, Z=None):
        '''
        dDims : data Dimensions
        pDims : projected Dimesions
        nClasses : num Classes
        tDepth : tree Depth
        
        Expected Dimensions:
        --------------------
        Bonsai Params // Optional
        
        W [numClasses*totalNodes, projectionDimension]
        V [numClasses*totalNodes, projectionDimension]
        Z [projectionDimension, dataDimension + 1]
        T [internalNodes, projectionDimension]

        internalNodes = 2**treeDepth - 1
        totalNodes = 2*internalNodes + 1

        sigma - tanh non-linearity
        sigmaI - Indicator function for node probabilities
        sigmaI - has to be set to infinity(1e9 for practicality)
        
        while doing testing/inference
        numClasses will be reset to 1 in binary case
        '''
        
        # Initialization of parameter variables
        
        self.dDims = dDims
        self.pDims = pDims

        # If number of classes is two we dont need to calculate other class probability
        if nClasses == 2:
            self.nClasses = 1
        else:
            self.nClasses = nClasses

        self.tDepth = tDepth
        self.sigma = sigma
        self.iNodes = 2**self.tDepth - 1
        self.tNodes = 2*self.iNodes + 1
        
        self.Z = tf.Variable(tf.random_normal([self.pDims, self.dDims]), name='Z', dtype=tf.float32)
        self.W = tf.Variable(tf.random_normal([self.nClasses * self.tNodes, self.pDims]), name='W', dtype=tf.float32)
        self.V = tf.Variable(tf.random_normal([self.nClasses * self.tNodes, self.pDims]), name='V', dtype=tf.float32)
        self.T = tf.Variable(tf.random_normal([self.iNodes, self.pDims]), name='T', dtype=tf.float32)
        
        
        self.assert_params()
        
        self.score = None
        self.X_ = None
        self.prediction = None
        
        
    def __call__(self, X, sigmaI):
        '''
        Function to build the Bonsai Tree graph
        
        Expected Dimensions
        -------------------
        X is [_, self.dDims]
        X_ is [_, self.pDims]
        '''
        errmsg = "Dimension Mismatch, X is [_, self.dataDimension]"
        assert (len(X.shape) == 2 and int(X.shape[1]) == self.dDims), errmsg
        
        
        # return score, X_ if exists where X_ is the projected X, i.e X_ = (Z.X)/(D^)
        if self.score is not None:
            return self.score, self.X_
        
        X_ = tf.divide(tf.matmul(self.Z, X, transpose_b=True),self.pDims) # dimensions are D^x1
        
        
        # For Root Node score...
        self.__nodeProb = [] # node probability list
        self.__nodeProb.append(1) # probability of x passing through root is 1.
        W_ = self.W[0:(self.nClasses)]# first K trees root W params : KxD^
        V_ = self.V[0:(self.nClasses)]# first K trees root V params : KxD^
        
        # All score sums variable initialized to root score... for each tree (Note: can be negative)
        score_ = self.__nodeProb[0]*tf.multiply(tf.matmul(W_, X_), tf.tanh(self.sigma * tf.matmul(V_, X_))) # : Kx1
        
        # Adding rest of the nodes scores...
        for i in range(1, self.tNodes):
            # current node is i
            # W, V of K different trees for current node
            W_ = self.W[i * self.nClasses:((i + 1) * self.nClasses)]# : KxD^
            V_ = self.V[i * self.nClasses:((i + 1) * self.nClasses)]# : KxD^
            
            # i's parent node shared theta param reshaping to 1xD^
            T_ = tf.reshape(self.T[int(np.ceil(i / 2.0) - 1.0)],[-1, self.pDims])# : 1xD^
            
            # Calculating probability that x should come to this node next given it is in parent node...
            prob = tf.divide((1 + ((-1)**(i + 1))*tf.tanh(tf.multiply(sigmaI, tf.matmul(T_, X_)))),2.0) # : scalar 1x1
            # Actual probability that x will come to this node...p(parent)*p(this|parent)...
            prob = self.__nodeProb[int(np.ceil(i / 2.0) - 1.0)] * prob # : scalar 1x1
            
            # adding prob to node prob list
            self.__nodeProb.append(prob)
            # New score addes to sum of scores...
            score_ += self.__nodeProb[i]*tf.multiply(tf.matmul(W_, X_), tf.tanh(self.sigma * tf.matmul(V_, X_))) # Kx1
            
            
        self.score = score_
        self.X_ = X_
        return self.score, self.X_
    
    
    def predict(self):
        '''
        Takes in a score tensor and outputs a integer class for each data point
        '''
        if self.prediction is not None:
            return self.prediction
        
        # If number of classes is two we dont need to calculate other class probability
        if self.nClasses > 2:
            # Finding argmax over first axis (k axis)
            self.prediction = tf.argmax(tf.transpose(self.score), 1) # score is kx1
        else:
            # Finding argmax over score and 0 score is 1x1
            self.prediction = tf.argmax(tf.concat([tf.transpose(self.score),0*tf.transpose(self.score)], 1), 1)
        return self.prediction

    def assert_params(self):
        
        # Asserting Initializaiton
        
        errRank = "All Parameters must has only two dimensions shape = [a, b]"
        assert len(self.W.shape) == len(self.Z.shape), errRank
        assert len(self.W.shape) == len(self.T.shape), errRank
        assert len(self.W.shape) == 2, errRank
        msg = "W and V should be of same Dimensions"
        assert self.W.shape == self.V.shape, msg
        errW = "W and V are [numClasses*totalNodes, projectionDimension]"
        assert self.W.shape[0] == self.nClasses * self.tNodes, errW
        assert self.W.shape[1] == self.pDims, errW
        errZ = "Z is [projectionDimension, dataDimension]"
        assert self.Z.shape[0] == self.pDims, errZ
        assert self.Z.shape[1] == self.dDims, errZ
        errT = "T is [internalNodes, projectionDimension]"
        assert self.T.shape[0] == self.iNodes, errT
        assert self.T.shape[1] == self.pDims, errT
        assert int(self.nClasses) > 0, "numClasses should be > 1"
        msg = "# of features in data should be > 0"
        assert int(self.dDims) > 0, msg
        msg = "Projection should be  > 0 dims"
        assert int(self.pDims) > 0, msg
        msg = "treeDepth should be >= 0"
        assert int(self.tDepth) >= 0, msg

In [155]:
class BonsaiTrainer():
    
    def __init__(self, tree, lW, lT, lV, lZ, lr, X, Y, sW, sV, sZ, sT):
        
        '''
        bonsaiObj - Initialised Bonsai Object and Graph...
        lW, lT, lV and lZ are regularisers to Bonsai Params...
        sW, sT, sV and sZ are sparsity factors to Bonsai Params...
        lr - learningRate fro optimizer...
        X is the Data Placeholder - Dims [_, dataDimension]
        Y - Label placeholder for loss computation
        useMCHLoss - For choice between HingeLoss vs CrossEntropy
        useMCHLoss - True - MultiClass - multiClassHingeLoss
        useMCHLoss - False - MultiClass - crossEntropyLoss
        '''
        #  Intializations of training parameters
        self.tree = tree
        
        # regularization params lambdas(l) (all are scalars)
        self.lW = lW
        self.lV = lV
        self.lT = lT
        self.lZ = lZ

        # sparsity parameters (scalars all...) will be used to calculate percentiles to make other cells zero
        self.sW = sW 
        self.sV = sV
        self.sT = sT
        self.sZ = sZ

        # placeholders for inputs and labels
        self.Y = Y # _ x nClasses
        self.X = X # _ x D
        
        # learning rate
        self.lr = lr
        
        # Asserting initialization
        self.assert_params()
        
        # place holder for path selection parameter sigmaI
        self.sigmaI = tf.placeholder(tf.float32, name='sigmaI')
        # invoking __call__ of tree getting initial values of score and projected X
        self.score, self.X_ = self.tree(self.X, self.sigmaI)
        # defining loss function tensorflow graph variables.....
        self.loss, self.marginLoss, self.regLoss = self.lossGraph()
        # defining single training step graph process ...
        self.tree.TrainStep = tf.train.AdamOptimizer(self.lr).minimize(self.loss)
        self.trainStep = self.tree.TrainStep
        # defining accuracy and prediction graph objects
        self.accuracy = self.accuracyGraph()
        self.prediction = self.tree.predict()
        
        
        # set all parameters above 0.99 if dont want to use IHT
        if self.sW > 0.99 and self.sV > 0.99 and self.sZ > 0.99 and self.sT > 0.99:
            self.isDenseTraining = True
        else:
            self.isDenseTraining = False
            
        # setting the hard thresholding graph obejcts
        self.hardThrsd()
        
    def hardThrsd(self):
        '''
        Set up for hard Thresholding Functionality
        '''
        # place holders for sparse parameters....
        self.__Wth = tf.placeholder(tf.float32, name='Wth')
        self.__Vth = tf.placeholder(tf.float32, name='Vth')
        self.__Zth = tf.placeholder(tf.float32, name='Zth')
        self.__Tth = tf.placeholder(tf.float32, name='Tth')

        # assigning the thresholded values to params as a graph object for tensorflow....
        self.__Woph = self.tree.W.assign(self.__Wth)
        self.__Voph = self.tree.V.assign(self.__Vth)
        self.__Toph = self.tree.T.assign(self.__Tth)
        self.__Zoph = self.tree.Z.assign(self.__Zth)

        # grouping the graph objects as one object....
        self.hardThresholdGroup = tf.group(
            self.__Woph, self.__Voph, self.__Toph, self.__Zoph)
        
    def hardThreshold(self, A, s):
        '''
        Hard thresholding function on Tensor A with sparsity s
        '''
        # copying to avoid errors....
        A_ = np.copy(A)
        # flattening the tensor...
        A_ = A_.ravel()
        if len(A_) > 0:
            # calculating the threshold value for sparse limit...
            th = np.percentile(np.abs(A_), (1 - s) * 100.0, interpolation='higher')
            # making sparse.......
            A_[np.abs(A_) < th] = 0.0
        # reconstructing in actual shape....
        A_ = A_.reshape(A.shape)
        return A_

    def accuracyGraph(self):
        '''
        Accuracy Graph to evaluate accuracy when needed
        '''
        if (self.tree.nClasses > 2):
            correctPrediction = tf.equal(tf.argmax(tf.transpose(self.score), 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))
        else:
            # some accuracy functional analysis for 2 classes could be different from this...
            y_ = self.Y * 2 - 1
            correctPrediction = tf.multiply(tf.transpose(self.score), y_)
            correctPrediction = tf.nn.relu(correctPrediction)
            correctPrediction = tf.ceil(tf.tanh(correctPrediction)) # final predictions.... round to(0 or 1)
            self.accuracy = tf.reduce_mean(
                tf.cast(correctPrediction, tf.float32))

        return self.accuracy
        
    
    def lossGraph(self):
        '''
        Loss Graph for given tree
        '''
        # regularization losses.....
        self.regLoss = 0.5 * (self.lZ * tf.square(tf.norm(self.tree.Z)) +
                          self.lW * tf.square(tf.norm(self.tree.W)) +
                          self.lV * tf.square(tf.norm(self.tree.V)) +
                          self.lT * tf.square(tf.norm(self.tree.T)))
        
        # emperical actual loss.....
        if (self.tree.nClasses > 2):
            '''
            Cross Entropy loss for MultiClass case in joint training for
            faster convergence
            '''
            # cross entropy loss....
            self.marginLoss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.transpose(self.score),
                                                               labels=tf.stop_gradient(self.Y)))
        else:
            # sigmoid loss....
            self.marginLoss = tf.reduce_mean(tf.nn.relu(1.0 - (2 * self.Y - 1) * tf.transpose(self.score)))
    
        # adding the losses...
        self.loss = self.marginLoss + self.regLoss
        return self.loss, self.marginLoss, self.regLoss
        
    def assert_params(self):
        # asserting the initialization....
        err = "sparsity must be between 0 and 1"
        assert self.sW >= 0 and self.sW <= 1, "W " + err
        assert self.sV >= 0 and self.sV <= 1, "V " + err
        assert self.sZ >= 0 and self.sZ <= 1, "Z " + err
        assert self.sT >= 0 and self.sT <= 1, "T " + err
        errMsg = "Dimension Mismatch, Y has to be [_, " + str(self.tree.nClasses) + "]"
        errCont = " numClasses are 1 in case of Binary case by design"
        assert (len(self.Y.shape) == 2 and self.Y.shape[1] == self.tree.nClasses), errMsg + errCont
        
        

        
    def train(self, batchSize, totalEpochs, sess, Xtrain, Xval, Ytrain, Yval, htc):
        iht = 0 # to keep a note if thresholding has been started ...
        numIters = Xtrain.shape[0] / batchSize # number of batches at a time...
        totalBatches = numIters * totalEpochs # total number of batch operations...
        treeSigmaI = 1 # controls the fidelity of the approximation too high can saturate tanh.
            
        maxTestAcc = -10000
        itersInPhase = 0
        
        for i in range(totalEpochs):
            print("\nEpoch Number: " + str(i))
            # defining training acc and loss
            trainAcc = 0.0
            trainLoss = 0.0
            
            numIters = int(numIters)
            
            for j in range(numIters):
                # creating batch.....sequentiall could be done randomly using choice function...
                mini_batchX = Xtrain[j*batchSize:(j+1)*batchSize,:] # B x D
                mini_batchY = Ytrain[j*batchSize:(j+1)*batchSize] # B x 
            
                # feed for training using tensorflow graph based gradient descent approach......
                _feed_dict = {self.X: mini_batchX, self.Y: mini_batchY,
                                  self.sigmaI: treeSigmaI}
                
                # training the tensorflow graph
                _, batchLoss, batchAcc = sess.run(
                    [self.trainStep, self.loss, self.accuracy],
                    feed_dict=_feed_dict)
                
                # calculating acc....
                trainAcc += batchAcc
                trainLoss += batchLoss
                
                # to update sigmaI.....
                if (itersInPhase % 100 == 0):
                    
                    # Making a random batch....
                    indices = np.random.choice(Xtrain.shape[0], 100)
                    rand_batchX = Xtrain[indices, :]
                    rand_batchY = Ytrain[indices, :]
                    rand_batchY = np.reshape(rand_batchY, [-1, self.tree.nClasses])

                    _feed_dict = {self.X: rand_batchX,
                                  self.sigmaI: treeSigmaI}
                    # Projected matrix...
                    Xcapeval = self.X_.eval(feed_dict=_feed_dict) # D^ x 1
                    # theta value... current...
                    Teval = self.tree.T.eval() # iNodes x D^
                    # current sum of all internal nodes sum(abs(theta^T.Z.x): iNoddes x miniBS) : 1x1
                    sum_tr = 0.0 
                    for k in range(0, self.tree.iNodes):
                        sum_tr += (np.sum(np.abs(np.dot(Teval[k], Xcapeval))))

                    
                    if(self.tree.iNodes > 0):
                        sum_tr /= (100 * self.tree.iNodes) # normalizing all sums
                        sum_tr = 0.1 / sum_tr # inverse of average sum
                    else:
                        sum_tr = 0.1
                    # thresholding inverse of sum as min(1000, sum_inv*2^(cuurent batch number / total bacthes / 30))
                    sum_tr = min(
                        1000, sum_tr * (2**(float(itersInPhase) /
                                            (float(totalBatches) / 30.0))))
                    # assiging higher values as convergence is reached...
                    treeSigmaI = sum_tr
                    
                itersInPhase+=1
                
                
                # to start hard thresholding after half_time(could vary) ......
                if((itersInPhase//numIters > htc*totalEpochs) and (not self.isDenseTraining)):
                    if(iht == 0):
                        print('\n\nHard Thresolding Started\n\n')
                        iht = 1
                    
                    # getting the current estimates of  W,V,Z,T...
                    currW = self.tree.W.eval()
                    currV = self.tree.V.eval()
                    currZ = self.tree.Z.eval()
                    currT = self.tree.T.eval()

                    # Setting a method to make some values of matrix zero....
                    self.__thrsdW = self.hardThreshold(currW, self.sW)
                    self.__thrsdV = self.hardThreshold(currV, self.sV)
                    self.__thrsdZ = self.hardThreshold(currZ, self.sZ)
                    self.__thrsdT = self.hardThreshold(currT, self.sT)

                    # runnign the hard thresholding graph....
                    fd_thrsd = {self.__Wth: self.__thrsdW, self.__Vth: self.__thrsdV,
                                self.__Zth: self.__thrsdZ, self.__Tth: self.__thrsdT}
                    sess.run(self.hardThresholdGroup, feed_dict=fd_thrsd)
                    
            
            
            print("Train Loss: " + str(trainLoss / numIters) +
                  " Train accuracy: " + str(trainAcc / numIters))
            
            # calculating the test accuracies with sigmaI as expected -> inf.. = 10^9
            oldSigmaI = treeSigmaI
            treeSigmaI = 1e9
            
            # test feed for tf...
            _feed_dict = {self.X: Xval, self.Y: Yval,
                                  self.sigmaI: treeSigmaI}
            
            # calculating losses....
            testAcc, testLoss, regTestLoss = sess.run([self.accuracy, self.loss, self.regLoss], feed_dict=_feed_dict)
            
            if maxTestAcc <= testAcc:
                maxTestAccEpoch = i
                maxTestAcc = testAcc
            
            print("Test accuracy %g" % testAcc)
            print("MarginLoss + RegLoss: " + str(testLoss - regTestLoss) +
                  " + " + str(regTestLoss) + " = " + str(testLoss) + "\n", end='\r')
#             time.sleep(0.1)
#             clear_output()
            
            
            treeSigmaI = oldSigmaI
            
        # sigmaI has to be set to infinity to ensure
        # only a single path is used in inference
        treeSigmaI = 1e9
        print("\nMaximum Test accuracy at compressed" +
              " model size(including early stopping): " +
              str(maxTestAcc) + " at Epoch: " +
              str(maxTestAccEpoch + 1) + "\nFinal Test" +
              " Accuracy: " + str(testAcc))

In [86]:
sess = tf.InteractiveSession()

In [211]:
tree = Bonsai(nClasses = nClasses, dDims = dDims, pDims = 3, tDepth = 2, sigma = 1.0)

X = tf.placeholder("float32", [None, dDims])
Y = tf.placeholder("float32", [None, nClasses])

bonsaiTrainer = BonsaiTrainer(tree, lW = 0.001, lT = 0.001, lV = 0.001, lZ = 0.0001, lr = 0.01, X = X, Y = Y,
                              sZ = 0.2999, sW = 0.2999, sV = 0.2999, sT = 0.2999)

In [212]:
sess.run(tf.global_variables_initializer())

In [213]:
totalEpochs = 50
batchSize = np.maximum(20, int(np.ceil(np.sqrt(Ytrain.shape[0]))))
bonsaiTrainer.train(batchSize, totalEpochs, sess, Xtrain, Xtest, Ytrain, Ytest, htc = 0)
# print('Time taken',end-start)


Epoch Number: 0


Hard Thresolding Started


Train Loss: 1.1960808277130126 Train accuracy: 0.3900000035762787
Test accuracy 0.342105
MarginLoss + RegLoss: 1.4425656 + 0.057546273 = 1.5001118

Epoch Number: 1
Train Loss: 1.0455241680145264 Train accuracy: 0.5700000047683715
Test accuracy 0.342105
MarginLoss + RegLoss: 1.2820411 + 0.05648101 = 1.3385221

Epoch Number: 2
Train Loss: 0.9584359288215637 Train accuracy: 0.6399999976158142
Test accuracy 0.342105
MarginLoss + RegLoss: 1.1473861 + 0.055320602 = 1.2027067

Epoch Number: 3
Train Loss: 0.8921676397323608 Train accuracy: 0.6399999976158142
Test accuracy 0.342105
MarginLoss + RegLoss: 1.0533763 + 0.054092903 = 1.1074692

Epoch Number: 4
Train Loss: 0.8498525857925415 Train accuracy: 0.6399999976158142
Test accuracy 0.342105
MarginLoss + RegLoss: 1.001907 + 0.052897226 = 1.0548042

Epoch Number: 5
Train Loss: 0.8248063206672669 Train accuracy: 0.6399999976158142
Test accuracy 0.342105
MarginLoss + RegLoss: 0.9829647 + 0.051801138 =

In [209]:
def calc_zero_ratios(tree):
    xZ = tree.Z.eval()
    xW = tree.W.eval()
    xV = tree.V.eval()
    xT = tree.T.eval()
    zs = np.sum(np.abs(xZ)>0.000000000000001)
    ws = np.sum(np.abs(xW)>0.000000000000001)
    vs = np.sum(np.abs(xV)>0.000000000000001)
    ts = np.sum(np.abs(xT)>0.000000000000001)
    print('Sparse ratios achieved...\nW:',ws,xW.shape,'\nV:',vs,xV.shape,'\nT:',ts,xT.shape,'\nZ:',zs,xZ.shape)
    _feed_dict = {bonsaiTrainer.X: Xtest, bonsaiTrainer.Y: Ytest,
                            bonsaiTrainer.sigmaI: 10e9}
    start = time.time()
    sess.run(bonsaiTrainer.tree.prediction, feed_dict=_feed_dict)
    end = time.time()
    print('Time taken :', end-start)

In [210]:
calc_zero_ratios(tree)

Sparse ratios achieved...
W: 44 (21, 3) 
V: 44 (21, 3) 
T: 6 (3, 3) 
Z: 8 (3, 4)
Time taken : 1.6680240631103516


# thankyou